# Thermal Analysis — Hand Calculations

Executable validation formulas for thermal and heat transfer analysis.
Enter your ANSYS results to compute analytical solutions and error.

**Reference:** `validation.md` — Thermal / Heat Transfer, CFD / Fluid Mechanics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def error_pct(ansys_val, hand_calc):
    return abs(ansys_val - hand_calc) / abs(hand_calc) * 100

def print_comparison(name, ansys_val, hand_calc, unit, threshold=5.0):
    err = error_pct(ansys_val, hand_calc)
    status = 'PASS' if err < threshold else 'FAIL'
    print(f'{name}:')
    print(f'  Hand calc:  {hand_calc:.4f} {unit}')
    print(f'  ANSYS:      {ansys_val:.4f} {unit}')
    print(f'  Error:      {err:.2f}% [{status} — threshold {threshold}%]')
    print()

---
## 1. 1D Conduction: q = kA(dT/dx)

In [ ]:
# --- INPUT ---
k = 50          # Thermal conductivity [W/m·K]
A = 0.01        # Cross-sectional area [m²]
L = 0.2         # Length [m]
T_hot = 100     # Hot face temperature [°C]
T_cold = 20     # Cold face temperature [°C]

ansys_heat_flux = 0.0   # <-- Enter ANSYS heat flux [W/m²]
ansys_temp_mid = 0.0    # <-- Enter ANSYS temperature at midpoint [°C]

# --- HAND CALC ---
dT = T_hot - T_cold
q_flux = k * dT / L                 # Heat flux [W/m²]
Q = q_flux * A                      # Total heat flow [W]
R_cond = L / (k * A)               # Thermal resistance [K/W]
T_mid = (T_hot + T_cold) / 2       # Linear profile midpoint

print(f'ΔT = {dT} °C')
print(f'q" = k·ΔT/L = {k}×{dT}/{L} = {q_flux:.0f} W/m²')
print(f'Q = q"·A = {Q:.1f} W')
print(f'R_cond = L/kA = {R_cond:.4f} K/W')
print(f'T_midpoint = {T_mid:.1f} °C (linear profile)')
print()

if ansys_heat_flux > 0:
    print_comparison('Heat flux', ansys_heat_flux, q_flux, 'W/m²')
if ansys_temp_mid > 0:
    print_comparison('Midpoint temp', ansys_temp_mid, T_mid, '°C')

---
## 2. Convection: q = hA(T_s - T_∞)

In [ ]:
# --- INPUT ---
h_conv = 25     # Convection coefficient [W/m²·K]
A_s = 0.05      # Surface area [m²]
T_surface = 80  # Surface temperature [°C]
T_inf = 25      # Ambient temperature [°C]

ansys_heat_rate = 0.0   # <-- Enter ANSYS heat transfer rate [W]

# --- HAND CALC ---
Q_conv = h_conv * A_s * (T_surface - T_inf)

print(f'Q = hA(T_s - T_∞) = {h_conv}×{A_s}×({T_surface}-{T_inf})')
print(f'Q = {Q_conv:.2f} W')
print()

if ansys_heat_rate > 0:
    print_comparison('Heat transfer rate', ansys_heat_rate, Q_conv, 'W')

---
## 3. Composite Wall (Series Resistances)

In [ ]:
# --- INPUT: layers from hot side to cold side ---
layers = [
    {'name': 'Steel',      'k': 50,   'L': 0.01},   # k [W/m·K], L [m]
    {'name': 'Insulation', 'k': 0.04, 'L': 0.05},
    {'name': 'Aluminum',   'k': 237,  'L': 0.005},
]
A_wall = 1.0          # Wall area [m²]
T_hot = 200           # Hot side [°C]
T_cold = 30           # Cold side [°C]

# Convection on each side (set to 0 to ignore)
h_hot = 100           # Hot side convection coeff [W/m²·K]
h_cold = 25           # Cold side convection coeff [W/m²·K]

# --- HAND CALC ---
R_total = 0
if h_hot > 0:
    R_conv_hot = 1 / (h_hot * A_wall)
    R_total += R_conv_hot
    print(f'R_conv_hot = 1/(h·A) = {R_conv_hot:.6f} K/W')

for layer in layers:
    R = layer['L'] / (layer['k'] * A_wall)
    R_total += R
    print(f'R_{layer["name"]} = L/kA = {R:.6f} K/W')

if h_cold > 0:
    R_conv_cold = 1 / (h_cold * A_wall)
    R_total += R_conv_cold
    print(f'R_conv_cold = 1/(h·A) = {R_conv_cold:.6f} K/W')

Q = (T_hot - T_cold) / R_total
print(f'\nR_total = {R_total:.6f} K/W')
print(f'Q = ΔT/R_total = {Q:.2f} W')

# Temperature at each interface
print('\nInterface temperatures:')
T_current = T_hot
if h_hot > 0:
    T_current -= Q * R_conv_hot
    print(f'  After hot convection: {T_current:.1f} °C')
for layer in layers:
    R = layer['L'] / (layer['k'] * A_wall)
    T_current -= Q * R
    print(f'  After {layer["name"]}: {T_current:.1f} °C')

---
## 4. Thermal Strain: ε = αΔT

In [ ]:
# --- INPUT ---
alpha = 12e-6   # Coefficient of thermal expansion [1/K] (steel ≈ 12e-6)
E = 200e9       # Young's modulus [Pa]
T_ref = 22      # Reference (stress-free) temperature [°C]
T_final = 150   # Final temperature [°C]
L_0 = 1.0       # Original length [m]

ansys_strain = 0.0      # <-- Enter ANSYS thermal strain [-]
ansys_stress = 0.0      # <-- Enter ANSYS thermal stress [Pa] (if constrained)

# --- HAND CALC ---
dT = T_final - T_ref
eps_thermal = alpha * dT                # Free thermal strain
delta_L = eps_thermal * L_0             # Free expansion
sigma_thermal = E * alpha * dT          # Fully constrained thermal stress

print(f'ΔT = {dT} °C')
print(f'Free thermal strain: ε = αΔT = {eps_thermal:.6f}')
print(f'Free expansion: ΔL = {delta_L*1000:.4f} mm')
print(f'Fully constrained stress: σ = EαΔT = {sigma_thermal/1e6:.1f} MPa')
print()

if ansys_strain > 0:
    print_comparison('Thermal strain', ansys_strain, eps_thermal, '—')
if ansys_stress > 0:
    print_comparison('Thermal stress', ansys_stress/1e6, sigma_thermal/1e6, 'MPa')

---
## 5. Pipe Flow: Hagen-Poiseuille

In [ ]:
# --- INPUT ---
rho = 998       # Fluid density [kg/m³] (water at 20°C)
mu = 1.002e-3   # Dynamic viscosity [Pa·s]
D = 0.05        # Pipe diameter [m]
L = 1.0         # Pipe length [m]
V_avg = 0.5     # Average velocity [m/s]

ansys_dp = 0.0          # <-- Enter ANSYS pressure drop [Pa]
ansys_v_center = 0.0    # <-- Enter ANSYS centerline velocity [m/s]

# --- HAND CALC ---
Re = rho * V_avg * D / mu
print(f'Re = ρVD/μ = {Re:.0f}')
print(f'Flow regime: {"Laminar" if Re < 2300 else "Turbulent"}')
print()

if Re < 2300:
    # Hagen-Poiseuille
    f = 64 / Re                                  # Friction factor (laminar)
    dP = f * (L / D) * (rho * V_avg**2 / 2)    # Darcy-Weisbach
    V_center = 2 * V_avg                         # Parabolic profile: V_max = 2·V_avg

    print(f'f = 64/Re = {f:.6f}')
    print(f'ΔP = f(L/D)(ρV²/2) = {dP:.2f} Pa')
    print(f'V_centerline = 2·V_avg = {V_center:.3f} m/s')
    print()

    if ansys_dp > 0:
        print_comparison('Pressure drop', ansys_dp, dP, 'Pa', threshold=10.0)
    if ansys_v_center > 0:
        print_comparison('Centerline velocity', ansys_v_center, V_center, 'm/s', threshold=5.0)
else:
    print('Turbulent flow — use Moody chart or Colebrook equation for f.')

---
## 6. Flat Plate Boundary Layer (Blasius)

In [ ]:
# --- INPUT ---
rho = 1.225     # Air density [kg/m³] at 20°C
mu = 1.81e-5    # Air dynamic viscosity [Pa·s]
V_inf = 10      # Freestream velocity [m/s]
x = 0.5         # Distance from leading edge [m]

ansys_delta = 0.0       # <-- Enter ANSYS BL thickness [m]
ansys_cf = 0.0          # <-- Enter ANSYS skin friction coefficient [-]

# --- HAND CALC ---
Re_x = rho * V_inf * x / mu
print(f'Re_x = ρV∞x/μ = {Re_x:.0f}')
print(f'Flow: {"Laminar" if Re_x < 5e5 else "Turbulent"}')
print()

# Blasius solution (laminar)
delta = 5 * x / np.sqrt(Re_x)                          # BL thickness
cf = 0.664 / np.sqrt(Re_x)                             # Local skin friction
tau_w = cf * 0.5 * rho * V_inf**2                      # Wall shear stress

print(f'δ = 5x/√Re_x = {delta*1000:.3f} mm')
print(f'c_f = 0.664/√Re_x = {cf:.6f}')
print(f'τ_w = c_f·½ρV² = {tau_w:.4f} Pa')
print()

if ansys_delta > 0:
    print_comparison('BL thickness', ansys_delta*1000, delta*1000, 'mm', threshold=10.0)
if ansys_cf > 0:
    print_comparison('Skin friction', ansys_cf, cf, '—', threshold=10.0)